In [2]:
import os
import json
import argparse
import logging
import pickle
import random

import pandas as pd
import torch
from torch.utils import data
import lightning.pytorch as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger

from proteinclip import data_utils, fasta_utils, swissprot, hparams
from proteinclip import contrastive

/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.34s/it]
Device set to use cuda:0


In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '../'))

In [21]:
args = {
    'unitnorm': True,
    'batch_size': 512,
    'dim': 128,
    'nhidden': 1,
    'learning_rate':1e-4,
    'out':'training',
    'name':'testrun',
    'max_epochs': 1000,
    
}

In [4]:
pl.seed_everything(seed=6489)


[rank: 0] Seed set to 6489


6489

In [5]:
keys_df = pd.read_parquet('../protclip_embed_dataset.parquet')

FileNotFoundError: [Errno 2] No such file or directory: '../protclip_embed_dataset.parquet'

In [1]:
from esm import pretrained
import torch
import pandas as pd
import numpy as np

# Load ESM2-33 model
esm_model, alphabet = pretrained.esm2_t33_650M_UR50D()
esm_model.eval()
batch_converter = alphabet.get_batch_converter()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
esm_model.to(device)

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [4]:
def embed_batch_esm2(seqs, repr_layer=33, device="cuda"):
    batch_data = [(f"seq{i}", seq) for i, seq in enumerate(seqs)]
    _, _, batch_tokens = batch_converter(batch_data)
    batch_tokens = batch_tokens.to(device)

    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[repr_layer], return_contacts=False)
    
    embeddings = []
    for i in range(len(seqs)):
        # Extract per-token representations and average (skip BOS and EOS)
        token_reps = results["representations"][repr_layer][i, 1:-1]
        avg_emb = token_reps.mean(0).cpu().numpy()
        embeddings.append(avg_emb)

    return embeddings


In [7]:
esm_embs = []
for i in range(0,len(keys_df),32):
    batch_seqs = keys_df['sequence'][i:i+32]
    batch_esm_embs = embed_batch_esm2(batch_seqs, device=device)
    esm_embs.append(batch_esm_embs)

esm_embs = np.concatenate(esm_embs,axis=0)


In [10]:
keys_df['ESM_embed'] = esm_embs.tolist()

In [6]:
keys_df = pd.read_parquet('text+ESM+protclip.parquet')

In [20]:
import gc
torch.cuda.empty_cache()
gc.collect()


3041

In [8]:
text_embed = dict(zip(
    keys_df["id"],
    keys_df["textual_embedding"]
))

In [7]:
esm_embed = dict(zip(
    keys_df["id"],
    keys_df["ESM_embed"]
))

In [9]:
shared_keys = sorted(set(text_embed.keys()).intersection(set(esm_embed.keys())))


In [10]:
random.seed(42)  # Set your desired seed here
random.shuffle(shared_keys)


In [11]:
dset = data_utils.CLIPDataset(
    pairs=shared_keys,
    map1=esm_embed,
    map2=text_embed,
    enforce_unit_norm=args['unitnorm'],
)

Checking for missing/zero embeddings: 100%|██████████| 10000/10000 [00:00<00:00, 29169.37it/s]
INFO:root:Trimmed 0 pairs with missing/zero embeddings, 10000 remain.


In [12]:
split_indices = data_utils.random_split(len(dset), [0.9, 0.05, 0.05])
dset_splits = [data.Subset(dset, idx) for idx in split_indices]

In [22]:
# Create data loaders
train_dl, valid_dl, _test_dl = [
    data.DataLoader(
        ds,
        batch_size=args['batch_size'],
        shuffle=(i == 0),
        # drop_last=(i == 0),
        num_workers=8,
        pin_memory=True,
    )
    for i, ds in enumerate(dset_splits)
]


In [23]:
# Define network
input_dim_1 = next(iter(train_dl))["x_1"].shape[-1]
input_dim_2 = next(iter(train_dl))["x_2"].shape[-1]
model_class = (
    contrastive.ContrastiveEmbedding
)
net = model_class(
    input_dim_1=input_dim_1,
    input_dim_2=input_dim_2,
    shared_dim=args['dim'],
    num_hidden=args['nhidden'],
    lr=args['learning_rate'],
)

In [24]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(
    monitor='val_loss',         # metric name to monitor
    patience=50,                 # epochs with no improvement after which to stop
    mode='min',                 # 'min' if you want to minimize the metric
    verbose=True                # optional: prints a message when triggered
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=1,
    mode='min',
    filename='best-checkpoint'
)


In [25]:
# Train
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="cuda",
    devices=1,
    enable_progress_bar=True,
    # logger=logger,
    # log_every_n_steps=10,
    deterministic=True,
    callbacks=[early_stop_callback, checkpoint_callback],

)
trainer.fit(net, train_dataloaders=train_dl, val_dataloaders=valid_dl)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params | Mode 
----------------------------------------------
0 | project_1    | MLP  | 1.8 M  | train
1 | project_2    | MLP  | 1.2 M  | train
  | other params | n/a  | 1      | n/a  
----------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.956    Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 0: 100%|██████████| 18/18 [00:00<00:00, 61.64it/s, v_num=2509882, train_loss=4.930, val_loss=5.400]

Metric val_loss improved. New best score: 5.399


Epoch 1: 100%|██████████| 18/18 [00:00<00:00, 29.80it/s, v_num=2509882, train_loss=4.620, val_loss=5.110]

Metric val_loss improved by 0.285 >= min_delta = 0.0. New best score: 5.114


...

Epoch 795: 100%|██████████| 18/18 [00:00<00:00, 33.70it/s, v_num=2509882, train_loss=0.345, val_loss=3.590]

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 3.590


Epoch 845: 100%|██████████| 18/18 [00:00<00:00, 32.93it/s, v_num=2509882, train_loss=0.357, val_loss=3.630]

Monitored metric val_loss did not improve in the last 50 records. Best score: 3.590. Signaling Trainer to stop.


Epoch 845: 100%|██████████| 18/18 [00:00<00:00, 32.84it/s, v_num=2509882, train_loss=0.357, val_loss=3.630]


In [26]:
# Export model as ONNX files
contrastive.model_to_onnx(
    net.project_1,
    "pc_project_1_10000.onnx",
    input_shape=(input_dim_1,),
)
contrastive.model_to_onnx(
    net.project_2,
    "pc_project_2_10000.onnx",
    input_shape=(input_dim_2,),
)


2025-04-24 18:21:15.976493381 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 1944038, index: 3, mask: {32, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 18:21:15.976506180 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 1944068, index: 32, mask: {25, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 18:21:15.976524545 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 1944067, index: 31, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-24 18:21:15.976531359 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 1944079, index: 42, mask: {53, }, error code: 22 error msg: Invalid argument. Specify